In [120]:
import pandas as pd
import pulp as p
import pickle

In [121]:
bidDetails = pd.read_csv('./output/bids_details.csv')
bidDetails

,ItemId,CommodityId,SupplierId,Quantity,Quoted_Price,ExtendedPrice
0,1899326,432118,100,100,85,850
1,1899326,432118,SUPPLIERRPC06,100,92,920
2,1899326,432118,SUPPLIERRPC05,100,95,950


In [122]:
bids = bidDetails.query('ItemId == 1899326 and Quoted_Price > 0')
bids = bids.reset_index()
bids

,index,ItemId,CommodityId,SupplierId,Quantity,Quoted_Price,ExtendedPrice
0,0,1899326,432118,100,100,85,850
1,1,1899326,432118,SUPPLIERRPC06,100,92,920
2,2,1899326,432118,SUPPLIERRPC05,100,95,950


In [123]:
# https://stats.stackexchange.com/questions/281162/scale-a-number-between-a-range
minBidPrice = bids['Quoted_Price'].min()
print(minBidPrice)
maxBidPrice = bids['Quoted_Price'].max()
print(maxBidPrice)

85
95


In [124]:
minRange = 1.0
maxRange = 10.0
bids['ScaledQuotedPrice'] = round((bids['Quoted_Price'] - minBidPrice)/(maxBidPrice - minBidPrice) * (maxRange - minRange) + minRange, 2)

In [125]:
modelFile = open('./selected_ai_ml_model.mdl', 'rb')
model = pickle.load(modelFile)
modelFile.close()

model

DecisionTreeRegressor()

In [126]:
supplierMapFile = open('./suppliermap.dict', 'rb')
supplierMap = pickle.load(supplierMapFile)
supplierMapFile.close()
supplierMap

{'100': 10001, 'SUPPLIERRPC06': 10002, 'SUPPLIERRPC05': 10003}

In [127]:
for index, row in bids.iterrows():
    commodityId = row['CommodityId']
    supplierId = supplierMap[row['SupplierId']]
    price = row['Quoted_Price']
    print("(" + str(row['SupplierId']) + ", " + str(commodityId) + ", " + str(price) + ")")
    
    bidData = pd.DataFrame([[supplierId, commodityId, price]])
    predict_bid = model.predict(bidData)
    
    quality = predict_bid[0][0]
    timeliness = predict_bid[0][1]
    
    print("Quality = " + str(quality) + ", Timeliness = " + str(timeliness))
    print("------------------------------------------------------------------")

    bids.at[index,'Quality'] = quality
    bids.at[index, 'Timeliness'] = timeliness

bids

(100, 432118, 85)
Quality = 9.0, Timeliness = 6.0
------------------------------------------------------------------
(SUPPLIERRPC06, 432118, 92)
Quality = 9.0, Timeliness = 6.0
------------------------------------------------------------------
(SUPPLIERRPC05, 432118, 95)
Quality = 9.0, Timeliness = 6.0
------------------------------------------------------------------


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


,index,ItemId,CommodityId,SupplierId,Quantity,Quoted_Price,ExtendedPrice,ScaledQuotedPrice,Quality,Timeliness
0,0,1899326,432118,100,100,85,850,1.0,9.0,6.0
1,1,1899326,432118,SUPPLIERRPC06,100,92,920,7.3,9.0,6.0
2,2,1899326,432118,SUPPLIERRPC05,100,95,950,10.0,9.0,6.0


In [168]:
'''
 Item selected in UI
 Qty is bid quantity
 Price entered in UI
 Call prediction for a item/price to get supplier characteristics
'''
problem = p.LpProblem(name='Supplier_Optimization', sense=p.LpMinimize)
problem

Supplier_Optimization:
MINIMIZE
None
VARIABLES

In [169]:
# Decision Variables
suppliers = p.LpVariable.dicts("", bids['SupplierId'], cat='Integer', lowBound=0, upBound=100)
suppliers

{'100': _100, 'SUPPLIERRPC06': _SUPPLIERRPC06, 'SUPPLIERRPC05': _SUPPLIERRPC05}

In [170]:
# Optimization function
priceProblem = p.lpSum(suppliers[bids['SupplierId'][i]] * bids['ScaledQuotedPrice'][i] for i in range(bids['SupplierId'].count()))
priceProblem

1.0*_100 + 10.0*_SUPPLIERRPC05 + 7.3*_SUPPLIERRPC06 + 0.0

In [171]:
#Call prediction for each supplier to get quality and timeliness
qualityProblem = p.lpSum(suppliers[bids['SupplierId'][i]] * bids['Quality'][i] for i in range(bids['SupplierId'].count()))
print(qualityProblem)
timelinessProblem =  p.lpSum(suppliers[bids['SupplierId'][i]] * bids['Timeliness'][i] for i in range(bids['SupplierId'].count()))
print(timelinessProblem)

9.0*_100 + 9.0*_SUPPLIERRPC05 + 9.0*_SUPPLIERRPC06
6.0*_100 + 6.0*_SUPPLIERRPC05 + 6.0*_SUPPLIERRPC06


In [172]:
problem += (1 * priceProblem) - (1 * qualityProblem) + (1 * timelinessProblem)
problem

Supplier_Optimization:
MINIMIZE
-2.0*_100 + 7.0*_SUPPLIERRPC05 + 4.3*_SUPPLIERRPC06 + 0.0
VARIABLES
0 <= _100 <= 100 Integer
0 <= _SUPPLIERRPC05 <= 100 Integer
0 <= _SUPPLIERRPC06 <= 100 Integer

In [173]:
# Constraints
# Required Quantity
problem += p.lpSum(suppliers[bids['SupplierId'][i]] for i in range(bids['SupplierId'].count())) == bids['Quantity'].unique()

problem

Supplier_Optimization:
MINIMIZE
-2.0*_100 + 7.0*_SUPPLIERRPC05 + 4.3*_SUPPLIERRPC06 + 0.0
SUBJECT TO
_C1: _100 + _SUPPLIERRPC05 + _SUPPLIERRPC06 = 100

VARIABLES
0 <= _100 <= 100 Integer
0 <= _SUPPLIERRPC05 <= 100 Integer
0 <= _SUPPLIERRPC06 <= 100 Integer

In [174]:
# Solve problem
problem.solve()

# Print status
status = p.LpStatus[problem.status]
print("Status:", status)

# Print optimal values of decision variables

supplier = []
selectedSupplier = ""
for v in problem.variables():
    print(v.name, "=", v.varValue)
    supplier.append(v.name.replace("_", "") + " = " + str(v.varValue))
    if v.varValue is not None and v.varValue > 0:
        selectedSupplier = v.name
quotedPrice = p.value(problem.objective)
print("Quoted Price : ", quotedPrice)

supplier = "Optimal value for supplier's " + "'" + str(supplier) + "' and the optimized objective function value is " + str(quotedPrice)
print(supplier)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/t4/204xh5g57vv3c7mjndmh4y7r0000gn/T/ef492c3aa0324d6388913b636eae6e81-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/t4/204xh5g57vv3c7mjndmh4y7r0000gn/T/ef492c3aa0324d6388913b636eae6e81-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 19 RHS
At line 21 BOUNDS
At line 25 ENDATA
Problem MODEL has 1 rows, 3 columns and 3 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is -200 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -200 to -1.79769e+308
Probing was tried 0 times and creat